In [1]:
from lib.config import Config
from lib.data_set import Dataset
from lib.model import NNModel
from lib import utils

In [2]:
import ctypes
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras import layers as KL
from tensorflow.keras import models as KM
from tensorflow import keras
import tensorflow as tf

## File name read

In [3]:
# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)
player = ctypes.windll.kernel32

ind = df2.iloc[1].isna()
files = np.concatenate([np.array(df.columns),np.array('normal/'+df2.columns[ind])])

## Configuration

In [4]:
# Override the base class of Config and Features for RNN_CNN Model
class RCNN_Config(Config):
    
    NAME = 'RNN_CNN'
    NUM_CLASSES = 2
    EPOCHS = 500
    BATCH_SIZE = 32
    COST_SENSITIVE = False
    CLASS_WEIGHTS = None
    TEST_FILES = files[[6,30,31,32,33,34,35]]
    
    FN_LP = 300
    FN_HP = None
    FN_IR = False
    DETREND_LAMBDA = 50

    DROP_WITH_ZSCORE = None
    
class Rect_dataset(Dataset):
    
    def __init__(self,config):
        super(Rect_dataset,self).__init__(config)
        self.n_env = config.N_ENV
    
    def rectify_data(self):
        
        self.Xr = utils.rectify_emg_moving_average(self.X,self.n_env)
        self.Xr2 = utils.rectify_emg_moving_average(self.X2,self.n_env)
        self.Xr3 = utils.rectify_emg_moving_average(self.X3,self.n_env)
    
    @property
    def train_set(self):
        return self.Xr,self.Y,self.F
    
    @property
    def valid_set(self):
        return self.Xr2,self.Y2,self.F2
    
    @property
    def test_set(self):
        return self.Xr3,self.Y3,self.F3

In [5]:
# Generate CWT-CNN configuration
config = RCNN_Config()
config.display()


Configurations:
BATCH_SIZE                     32
CHANNELS                       ['LEFT_TA', 'LEFT_TS', 'LEFT_BF', 'LEFT_RF', 'RIGHT_TA', 'RIGHT_TS', 'RIGHT_BF', 'RIGHT_RF']
CLASS_WEIGHTS                  None
COST_SENSITIVE                 False
DETREND_LAMBDA                 50
DROP_WITH_ZSCORE               None
EPOCHS                         500
FN_HP                          None
FN_IR                          False
FN_LP                          300
NAME                           RNN_CNN
NUM_CLASSES                    2
N_ENV                          20
RECT                           False
REMOVE_FREQS                   True
SAME_LABEL                     True
SAVE                           False
SCALE                          True
SHUFFLE                        True
STEP_SIZE                      512
TEST_FILES                     ['G08_FoG_1_trial_1_emg.csv' 'normal/G09_Walking_trial_2_emg.csv'
 'normal/G09_Walking_trial_4_emg.csv' 'normal/G09_Walking_trial_6_emg.csv'
 'normal

## Data generate

In [6]:
data = Rect_dataset(config)

In [7]:
# Load data from files
data.load_data(files)

# Rectify the data to get envelope
data.rectify_data()

X_train,Y_train,F1 = data.train_set
X_valid,Y_valid,F2 = data.valid_set
X_test, Y_test, F3 = data.test_set

skip
skip
3/174: G06_FoG_trial_1_emg.csv
4/174: G06_FoG_trial_2_emg.csv
5/174: G06_FoG_trial_3_emg.csv
6/174: G07_Freezing_Trial1_trial_1_emg.csv
7/174: G08_FoG_1_trial_1_emg.csv
8/174: G08_FoG_2_trial_1_emg.csv
9/174: G11_FoG_trial_1_emg.csv
10/174: G11_FoG_trial_2_emg.csv
11/174: P379_M050_2_OFF_A_FoG_trial_1_emg.csv
12/174: P379_M050_2_OFF_A_FoG_trial_2_emg.csv
13/174: P379_M050_2_OFF_A_FoG_trial_3_emg.csv
14/174: P379_M050_2_OFF_B_FoG_trial_1_emg.csv
15/174: P379_M050_2_OFF_B_FoG_trial_2_emg.csv
16/174: P379_M050_2_OFF_B_FoG_trial_3_emg.csv
17/174: P551_M050_2_A_FoG_trial_1_emg.csv
18/174: P551_M050_2_B_FoG_trial_1_emg.csv
19/174: P551_M050_2_B_FoG_trial_2_emg.csv
20/174: P812_M050_2_B_FoG_trial_1_emg.csv
21/174: P812_M050_2_B_FoG_trial_2_emg.csv
22/174: normal/G02_Walking_trial_1_emg.csv
23/174: normal/G03_Walking_trial_1_emg.csv
24/174: normal/G03_Walking_trial_2_emg.csv
25/174: normal/G05_Walking_struct_fixed_trial_1_emg.csv
26/174: normal/G05_Walking_struct_fixed_trial_2_emg.cs

171/174: normal/P940_MSham_A_Walking_trial_6_emg.csv
172/174: normal/P940_MSham_B_Walking_trial_2_emg.csv
173/174: normal/P940_MSham_B_Walking_trial_4_emg.csv
174/174: normal/P940_MSham_B_Walking_trial_6_emg.csv


## Model

In [12]:
# Override base class of SimpleMode for RNN_CNN
class RNN_CNN_Model(NNModel):
    
    def build(self,config):
        self.input_shape = [config.WINDOW_SIZE, len(config.CHANNELS)]
        kernel_size=3
        reg=keras.regularizers.l2(1e-4)
        drop_rate = 0.
        kernel_initializer = 'glorot_normal'
        mo = 0.8
        st = 1
        axis = 2
        model = KM.Sequential()
        model.add(KL.InputLayer(input_shape=self.input_shape))

        model.add(KL.Bidirectional(KL.LSTM(32,return_sequences=True,
                                                   recurrent_regularizer=reg)))

        model.add(KL.Conv1D(filters=32, kernel_size=kernel_size,strides=st,
                                padding='same',
                                kernel_regularizer=reg,
                               ))
        model.add(KL.BatchNormalization(momentum=mo))

        model.add(KL.LeakyReLU(0.1))
        model.add(KL.MaxPooling1D(2))
        model.add(KL.Dropout(drop_rate))
        model.add(KL.Conv1D(filters=16, kernel_size=kernel_size,strides=st,
                                padding='same',
                                kernel_regularizer=reg,
                               ))
        model.add(KL.BatchNormalization(momentum=mo))

        model.add(KL.LeakyReLU(0.1))
        model.add(KL.MaxPooling1D(2))
        model.add(KL.Dropout(drop_rate))
        model.add(KL.Conv1D(filters=8, kernel_size=kernel_size,strides=st,
                                padding='same',
                                kernel_regularizer=reg,
                               ))
        model.add(KL.BatchNormalization(momentum=mo))

        model.add(KL.LeakyReLU(0.1))
        model.add(KL.MaxPooling1D(2))
        model.add(KL.Dropout(drop_rate))
        model.add(KL.Conv1D(filters=4, kernel_size=kernel_size,strides=st,
                                padding='same',
                                kernel_regularizer=reg,
                               ))
        model.add(KL.BatchNormalization(momentum=mo))

        model.add(KL.LeakyReLU(0.1))
        model.add(KL.MaxPooling1D(2))

        model.add(KL.GlobalAveragePooling1D())
        model.add(KL.Dropout(drop_rate))
        model.add(KL.Dense(config.NUM_CLASSES,activation='softmax',kernel_regularizer=reg))
        
        model.summary()
        if config.COST_SENSITIVE:
            self.cost_matrix = config.COST_MATRIX
            model.compile(loss=self.sparse_cost_sensitive_loss, optimizer="adam", metrics=['accuracy'])
            print('Using cost sensitive with cost matrix:\n',np.array(self.cost_matrix))
        else:
            model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
            if config.CLASS_WEIGHTS != None:
                print('Using categorical crossentropy with class weights:\n',config.CLASS_WEIGHTS)
            else:
                print('Using categorical crossentropy without class weights.')
        
        return model

    def sparse_cost_sensitive_loss (self,y_true,y_pred):
        cost_matrix = self.cost_matrix
        batch_cost_matrix = tf.nn.embedding_lookup(cost_matrix, tf.argmax(y_true,axis=1))
        eps = 1e-6
        probability = tf.clip_by_value(y_pred, eps, 1-eps)
        cost_values = tf.math.log(1-probability)*batch_cost_matrix
        loss = tf.reduce_mean(-tf.reduce_sum(cost_values, axis=1))
        return loss
    
    def model_metrics(self,data,label):
        pred = self.keras_model.predict(data)
        acc = metrics.accuracy_score(np.argmax(label,axis=1),np.argmax(pred,axis=1))
        cm = metrics.confusion_matrix(np.argmax(label,axis=1),np.argmax(pred,axis=1))
        f1 = metrics.f1_score(np.argmax(label,axis=1),np.argmax(pred,axis=1),average='macro')
        return acc,cm,f1

## Data split

In [13]:
# data split and processing for model
class_id = [1,2,6]
binary = False
x_train,y_train,x_valid,y_valid,x_test,y_test,oh = utils.data_split_oh((X_train,X_valid,X_test),
                                                                    (Y_train,Y_valid,Y_test),
                                                                    class_id,
                                                                    binary,
                                                                    random_state = 555)

## Model training

In [14]:
config.COST_MATRIX = tf.constant([[0,1.],
              [10,0]])

if binary:
    config.COST_SENSITIVE = True
    config.NUM_CLASSES = 2
else:
    config.COST_SENSITIVE = False
    config.NUM_CLASSES = len(class_id)

rcnn_model = RNN_CNN_Model('RNN_CNN',config,'./model/RNN_CNN/')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 1024, 64)          10496     
_________________________________________________________________
conv1d (Conv1D)              (None, 1024, 32)          6176      
_________________________________________________________________
batch_normalization (BatchNo (None, 1024, 32)          128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 1024, 32)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 512, 32)           0         
_________________________________________________________________
dropout (Dropout)            (None, 512, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 512, 16)           1

In [15]:
early_stopping = keras.callbacks.EarlyStopping(patience = 20,
                                             monitor = 'val_loss', 
                                             #baseline = 0.9,
                                             restore_best_weights=True)
rcnn_model.train((x_train,y_train),(x_valid,y_valid),config.EPOCHS,config.BATCH_SIZE,[early_stopping])


Starting at epoch 0.

Checkpoint Path: ./model/RNN_CNN/rnn_cnn20211104T2142\RNN_CNN_rnn_cnn_{epoch:04d}.h5
Epoch 1/500
14/14 [==============================] - 15s 654ms/step - loss: 1.0277 - accuracy: 0.5094 - val_loss: 0.9846 - val_accuracy: 0.6507
Epoch 2/500
14/14 [==============================] - 7s 527ms/step - loss: 0.9282 - accuracy: 0.5661 - val_loss: 0.8506 - val_accuracy: 0.7055
Epoch 3/500
14/14 [==============================] - 7s 504ms/step - loss: 0.8644 - accuracy: 0.6495 - val_loss: 0.7682 - val_accuracy: 0.7123
Epoch 4/500
14/14 [==============================] - 7s 540ms/step - loss: 0.8238 - accuracy: 0.6793 - val_loss: 0.7591 - val_accuracy: 0.7740
Epoch 5/500
14/14 [==============================] - 7s 509ms/step - loss: 0.7646 - accuracy: 0.7631 - val_loss: 0.6865 - val_accuracy: 0.7603
Epoch 6/500
14/14 [==============================] - 7s 503ms/step - loss: 0.7543 - accuracy: 0.7150 - val_loss: 0.6811 - val_accuracy: 0.7671
Epoch 7/500
14/14 [==============

14/14 [==============================] - 8s 581ms/step - loss: 0.2424 - accuracy: 0.9851 - val_loss: 0.3707 - val_accuracy: 0.8630
Epoch 58/500
14/14 [==============================] - 8s 589ms/step - loss: 0.2205 - accuracy: 0.9818 - val_loss: 0.3724 - val_accuracy: 0.8767
Epoch 59/500
14/14 [==============================] - 9s 622ms/step - loss: 0.2185 - accuracy: 0.9843 - val_loss: 0.3815 - val_accuracy: 0.8836
Epoch 60/500
14/14 [==============================] - 8s 602ms/step - loss: 0.2079 - accuracy: 0.9824 - val_loss: 0.3757 - val_accuracy: 0.8699
Epoch 61/500
14/14 [==============================] - 7s 535ms/step - loss: 0.2299 - accuracy: 0.9786 - val_loss: 0.3563 - val_accuracy: 0.8973
Epoch 62/500
14/14 [==============================] - 9s 614ms/step - loss: 0.2056 - accuracy: 0.9785 - val_loss: 0.4124 - val_accuracy: 0.8356
Epoch 63/500
14/14 [==============================] - 8s 546ms/step - loss: 0.2059 - accuracy: 0.9852 - val_loss: 0.4113 - val_accuracy: 0.8836
Epoch

## Model evaluation

In [16]:
acc_train,cm_train,f1_train = rcnn_model.model_metrics(x_train,y_train)
acc_valid,cm_valid,f1_valid = rcnn_model.model_metrics(x_valid,y_valid)
acc_test,cm_test,f1_test = rcnn_model.model_metrics(x_test,y_test)
print('acc_train: %f\nf1_train: %f\nconfusion_matrix:\n'%(acc_train,f1_train),cm_train)
print('acc_valid: %f\nf1_valid: %f\nconfusion_matrix:\n'%(acc_valid,f1_valid),cm_valid)
print('acc_test: %f\nf1_test: %f\nconfusion_matrix:\n'%(acc_test,f1_test),cm_test)

acc_train: 0.984018
f1_train: 0.979078
confusion_matrix:
 [[ 83   0   2]
 [  0 246   0]
 [  2   3 102]]
acc_valid: 0.897260
f1_valid: 0.810269
confusion_matrix:
 [[ 9  6  4]
 [ 1 92  2]
 [ 1  1 30]]
acc_test: 0.778523
f1_test: 0.757871
confusion_matrix:
 [[22 12  1]
 [ 6 47  0]
 [12  2 47]]
